# Triton 관리형 온라인 엔드포인트 호출 및 예측 결과 가져오기

이 노트북에서는 이전 단계에서 생성한 엔드포인트를 사용합니다. Triton 관리형 온라인 엔드포인트가 생성되고 배포되어 있어야 합니다. 이 노트북의 셀을 성공적으로 실행하려면 REST 엔드포인트와 키가 필요합니다.

> 이 노트북은 Triton 엔드포인트를 사용하는 Azure Machine Learning 예제를 기반으로 합니다: https://github.com/Azure/azureml-examples/tree/main/cli/endpoints/online/triton/single-model

**03-deploy-model** 폴더에서 **triton_densenet_scoring.py** 스크립트를 찾을 수 있습니다. 엔드포인트와 키를 매개변수로 전달하여 이 스크립트를 실행합니다. 스크립트의 내용을 검토해 보세요.

스크립트는 이미지 URL을 입력으로 받아 엔드포인트에 전송하기 전에 전처리합니다. 결과는 같은 폴더에 저장된 **densenet_labels.txt** 파일을 사용하여 "변환"됩니다.

In [ ]:
# 스코어링 스크립트 실행에 필요한 라이브러리 설치
!pip install -q numpy tritonclient[http] pillow gevent

## 매개변수 업데이트 및 호출 실행

아래 코드 셀에서 값을 입력하세요.

- `endpoint_url`: Endpoint의 **Consume** 탭에 있는 REST endpoint
- `primary_key`: Endpoint의 **Consume** 탭에 있는 Primary key
- `image_url`: 테스트할 이미지 URL (기본값: 공작 이미지)

값 입력 후 셀을 실행하면 `triton_densenet_scoring.py`가 호출됩니다.

In [ ]:
import os
import sys
import glob
import subprocess
from pathlib import Path

endpoint_url = "<your-endpoint-url>"
primary_key = "<your-primary-key>"
image_url = "https://aka.ms/peacock-pic"

if endpoint_url.startswith("<") or primary_key.startswith("<"):
    raise ValueError("endpoint_url과 primary_key를 실제 값으로 변경한 뒤 다시 실행하세요.")

candidate_script_paths = [
    Path.cwd() / "triton_densenet_scoring.py",
    Path.home() / "cloudfiles" / "code" / "Users" / "*" / "azure-machine-learning-workshop" / "Notebooks" / "03-deploy-model" / "triton_densenet_scoring.py",
    Path.home() / "cloudfiles" / "code" / "Users" / "*" / "Notebooks" / "03-deploy-model" / "triton_densenet_scoring.py",
]

script_path = None
for candidate in candidate_script_paths:
    if "*" in str(candidate):
        matches = glob.glob(str(candidate))
        if matches:
            script_path = Path(matches[0])
            break
    elif candidate.exists():
        script_path = candidate
        break

if script_path is None:
    raise FileNotFoundError("triton_densenet_scoring.py를 찾을 수 없습니다. 03-deploy-model 폴더 경로를 확인하세요.")

print(f"Using script: {script_path}")

subprocess.run(
    [
        sys.executable,
        str(script_path),
        "--base_url", endpoint_url,
        "--token", primary_key,
        "--image_url", image_url,
    ],
    check=True,
)

다른 이미지를 테스트하려면 위 셀에 제공된 이미지 URL을 변경하면 됩니다. 원하는 이미지 URL을 자유롭게 사용하세요. 다른 이미지를 테스트할 수 있도록 미리 준비된 세 개의 URL이 있습니다:
- https://aka.ms/image01
- https://aka.ms/image02
- https://aka.ms/image03

## 리소스 삭제

실험이 끝나면:
- 엔드포인트를 삭제하세요
- 컴퓨팅 인스턴스를 중지하세요